In [1]:
from tensorflow.keras.models import load_model
from Commonfunctions import *
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
# model = load_model('3nd_best_model.h5')
model = load_model('all_frame_model.h5')

In [3]:
# DATA_PATH = os.path.join('DataSetw')

DATA_PATH = os.path.join('Sinhala_Word') #for 20 frame model

sign_word = [name for name in os.listdir(DATA_PATH) if os.path.isdir(os.path.join(DATA_PATH, name))]

sign_word = np.array(sign_word)

print(sign_word)

['no sign' 'none' 'ආසන්න වෙලා' 'ඉක්මනින්' 'ඉබ්බා' 'ඉලක්කයට' 'ඒ නිසා' 'ඔබ'
 'කපුටාට' 'කපුටෙක්' 'කෑමක්' 'කියලා' 'කේජු කෑල්ලක්' 'දවසක්' 'දිනුවා'
 'පරක්කු උණා' 'බඩගින්නෙන්' 'යන බව' 'යනවා' 'සිටියා' 'සිනහ උණා' 'සෙමෙන්'
 'සෙව්වා' 'හම්බුනා' 'හාවා' 'හාවාට' 'හාවෙක්' 'හිතනවට වඩා']


In [4]:
label_map = {label:num for num, label in enumerate(sign_word)}
label_map

{'no sign': 0,
 'none': 1,
 'ආසන්න වෙලා': 2,
 'ඉක්මනින්': 3,
 'ඉබ්බා': 4,
 'ඉලක්කයට': 5,
 'ඒ නිසා': 6,
 'ඔබ': 7,
 'කපුටාට': 8,
 'කපුටෙක්': 9,
 'කෑමක්': 10,
 'කියලා': 11,
 'කේජු කෑල්ලක්': 12,
 'දවසක්': 13,
 'දිනුවා': 14,
 'පරක්කු උණා': 15,
 'බඩගින්නෙන්': 16,
 'යන බව': 17,
 'යනවා': 18,
 'සිටියා': 19,
 'සිනහ උණා': 20,
 'සෙමෙන්': 21,
 'සෙව්වා': 22,
 'හම්බුනා': 23,
 'හාවා': 24,
 'හාවාට': 25,
 'හාවෙක්': 26,
 'හිතනවට වඩා': 27}

In [8]:
no_sequences=15
sequence_length = 89

In [9]:
sequences, labels = [], []
for action in sign_word:
    for sequence in range(no_sequences):
        window = []
        # for frame_num in range(60,sequence_length, 6):
        for frame_num in range(50,sequence_length): # for 20 frame model
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
# sign_word = ['ආසන්න වෙලා', 'බඩගින්නෙන්', 'දවසක්', 'දිනුවා', 'ඒ නිසා', 'හම්බුනා', 'හාවා',
#  'හාවාට', 'හාවෙක්', 'හිතනවට වඩා', 'ඉබ්බා', 'ඉක්මනින්', 'ඉලක්කයට',
#  'කපුටාට', 'කපුටෙක්', 'කේජු කෑල්ලක්', 'කෑමක්', 'කියලා', 'no sign', 'none',
#  'ඔබ', 'පරක්කු උණා', 'සෙමෙන්', 'සෙව්වා', ' සිනහ උණා', 'සිටියා', 'යන බව',
#  'යනවා']

In [10]:
np.array(sequences).shape

(420, 39, 1662)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
X = np.array(sequences)

In [13]:
res = model.predict(X)

14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step


In [14]:
sign_word[np.argmax(res[32])]

'ආසන්න වෙලා'

In [15]:
sign_word[np.argmax(y[32])]

'ආසන්න වෙලා'

In [16]:
ytrue = np.argmax(y, axis=1).tolist()
yhat = np.argmax(res, axis=1).tolist()

In [17]:
accuracy_score(ytrue, yhat)

0.9595238095238096

### For single word

In [21]:
cap = cv2.VideoCapture(0)

sequence =[]


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()
    display_text = 'READY'
    text_duration = 1  
    text_sequence = ['READY', '1', '2', '3', 'GO']
    text_index = 0

    while cap.isOpened():
        try:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            
            if time.time() - start_time > text_duration:
                text_index += 1
                start_time = time.time()

            
            if text_index < len(text_sequence):
                display_text = text_sequence[text_index]
            else:
                display_text = None

            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)

            
            if display_text:
                cv2.putText(image, display_text, (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 255), 4, cv2.LINE_AA)
            else:
                
                keypoints = extract_keypoints(results)
                # print(keypoints)
                sequence.append(keypoints)
                if len(sequence) >= 39:
                    break

                

            cv2.imshow('OpenCV Feed', image)    

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        except Exception as e:
            print("An error occurred:", e)
            pass
    cap.release()
    cv2.destroyAllWindows()

# print(len(sequence))
print('action captured')

window = []
# for frame_num in range(10,30, 6):
for frame_num in range(0,39): # for 20 frame model
    # print(frame_num)
    window.append(sequence[frame_num])

window =np.array(window)
reshaped_window = np.expand_dims(window, axis=0)

prediction =model.predict(np.array(reshaped_window))

print(sign_word[np.argmax(prediction)])



action captured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
හාවාට


### For sentence

In [22]:
sequence = []
sub_sequence =[]
sentence = []
predictions = []
threshold = 0.3

cap = cv2.VideoCapture(0)


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()
    display_text = 'READY'
    text_duration = 1  
    text_sequence = ['READY', '1', '2', '3', 'GO']
    text_index = 0

    while cap.isOpened():
        try:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            
            if time.time() - start_time > text_duration:
                text_index += 1
                start_time = time.time()

            
            if text_index < len(text_sequence):
                display_text = text_sequence[text_index]
            else:
                display_text = None

            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)

            
            if display_text:
                cv2.putText(image, display_text, (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 255), 4, cv2.LINE_AA)
            else:
                keypoints = extract_keypoints(results)
                # print(keypoints)
                sequence.append(keypoints)

            cv2.imshow('OpenCV Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break

        except Exception as e:
            print("An error occurred:", e)
            pass

# print(len(sequence))
print('action captured')

for i in range(len(sequence) - 39 + 1):
    sub_sequence = sequence[i:i + 39]

    if len(sub_sequence) < 39:
        continue

    window = [sub_sequence[frame_num] for frame_num in range(10, 30)]
    res = model.predict(np.expand_dims(window, axis=0))[0]

    if res[np.argmax(res)] > threshold:
        predictions.append(sign_word[np.argmax(res)])
        if len(predictions) >= 20 and np.unique(predictions[-20:])[0] == sign_word[np.argmax(res)]:
            if len(sentence) > 0:
                if sign_word[np.argmax(res)] != sentence[-1]:
                    sentence.append(sign_word[np.argmax(res)])
                    predictions = []
            else:
                sentence.append(sign_word[np.argmax(res)])
                predictions = []

print(sentence)

action captured
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 346ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[]


### For sentence with next

In [23]:
cap = cv2.VideoCapture(0)

sequence =[]
sentence = []

next_text_start_time = None
next_text_duration = 1 

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()
    display_text = 'READY'
    text_duration = 1  
    text_sequence = ['READY', '1', '2', '3', 'GO']
    text_index = 0

    while cap.isOpened():
        try:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            
            if time.time() - start_time > text_duration:
                text_index += 1
                start_time = time.time()

            
            if text_index < len(text_sequence):
                display_text = text_sequence[text_index]
            else:
                display_text = None

            image, results = mediapipe_detection(frame, holistic)
            draw_landmarks(image, results)

            
            if display_text:
                cv2.putText(image, display_text, (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 255), 4, cv2.LINE_AA)
            else:
                
                keypoints = extract_keypoints(results)
                # print(keypoints)
                sequence.append(keypoints)
                if len(sequence) >= 39:
                    # break
                    window = []
                    # for frame_num in range(10,30, 6):
                    for frame_num in range(10,30): # for 20 frame model
                        # print(frame_num)
                        window.append(sequence[frame_num])

                    window =np.array(window)
                    reshaped_window = np.expand_dims(window, axis=0)

                    prediction =model.predict(np.array(reshaped_window))[0]

                    # if prediction[np.argmax(prediction)] > 0.3:

                    print(sign_word[np.argmax(prediction)])

                    sentence.append(sign_word[np.argmax(prediction)])

                    # else:
                    #     print("not clear")
                    #     print(prediction[np.argmax(prediction)])



                    cv2.putText(image, "next", (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 255), 4, cv2.LINE_AA)
                    
                    next_text_start_time = time.time()
                    sequence =[]


            if next_text_start_time and (time.time() - next_text_start_time < next_text_duration):
                cv2.putText(image, "next", (200, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 255), 4, cv2.LINE_AA)
                

            cv2.imshow('OpenCV Feed', image)    

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        except Exception as e:
            print("An error occurred:", e)
            pass
    cap.release()
    cv2.destroyAllWindows()


print('action captured')
print(sentence)





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
සෙව්වා
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
සෙව්වා
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
none
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
සෙව්වා
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
none
action captured
['සෙව්වා', 'සෙව්වා', 'none', 'සෙව්වා', 'none']


In [ ]:
prediction =model.predict(np.array(reshaped_window))[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [ ]:
prediction

array([1.6746070e-03, 4.2386702e-03, 3.4010160e-04, 2.7251546e-03,
       8.0260597e-03, 6.6352682e-03, 5.6252358e-03, 1.9393887e-02,
       2.8515533e-01, 8.2902603e-02, 1.1482708e-02, 8.2599208e-02,
       7.3352404e-02, 8.2846900e-04, 7.6797669e-04, 6.8548089e-03,
       2.9821639e-04, 8.5034836e-03, 1.1603448e-03, 2.4806000e-03,
       3.3671760e-03, 2.4486028e-03, 3.5938218e-01, 1.3607995e-02,
       2.2413956e-03, 9.0061119e-03, 1.5495213e-03, 3.3519606e-03],
      dtype=float32)

In [ ]:
max(prediction)

0.35938218

In [ ]:
prediction[0][np.argmax(prediction)]

0.75223666

In [ ]:
cap.release()
cv2.destroyAllWindows()